Hệ thống nhận diện 4 loại bệnh trên lá lúa:
- Bacterial Blight (Bạc lá)
- Blast (Đạo ôn)
- Brown Spot (Đốm nâu)
- Twisted Draft (Xoắn lá)

Sử dụng YOLOv8 với Oriented Bounding Box (OBB) để nhận diện chính xác hơn.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics -q
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install opencv-python matplotlib numpy pillow tqdm psutil -q

In [ ]:
%%writefile config.py
# Path settings
DATA_ROOT = "/content/drive/My Drive/RiceLeafDisease"
DATA_YAML = f"{DATA_ROOT}/data.yaml"
TEST_IMAGES = f"{DATA_ROOT}/test/images"
SAVE_DIR = DATA_ROOT

# Training settings
MODEL_BASE = "yolov8n-obb.pt"  # mô hình nhỏ nhưng hiệu quả
PROJECT_DIR = "runs/train"
EXPERIMENT_NAME = "exp-obb-100"

# Training hyperparameters
EPOCHS = 100
IMAGE_SIZE = 640
BATCH_SIZE = 48     # điều chỉnh nếu gặp lỗi CUDA out of memory
DEVICE = 0
WORKERS = 12
CACHE = True        # tăng tốc quá trình huấn luyện

# Prediction settings
CONFIDENCE = 0.5    # ngưỡng tin cậy cho dự đoán
PRED_PROJECT = "runs/predict"
PRED_NAME = "test_results-obb-100"

# File paths for trained model
BEST_MODEL_PATH = f"{PROJECT_DIR}/{EXPERIMENT_NAME}/weights/best.pt"

In [ ]:
%%writefile data.yaml
train: /content/drive/My Drive/RiceLeafDisease/train/images
val: /content/drive/My Drive/RiceLeafDisease/valid/images
test: /content/drive/My Drive/RiceLeafDisease/test/images
nc: 4
names: ['Bacterial Blight', 'Blast', 'Brown Spot', 'Twisted Draft']

In [ ]:
%%writefile setup.py
import os
import subprocess
from google.colab import drive

def mount_drive():
    """Mount Google Drive to access dataset."""
    drive.mount('/content/drive')
    print("Drive mounted successfully!")

def check_dataset():
    """Check if the dataset is available."""
    data_dir = "/content/drive/My Drive/RiceLeafDisease"
    try:
        files = os.listdir(data_dir)
        print(f"Found {len(files)} items in dataset folder")
        return True
    except FileNotFoundError:
        print("Dataset folder not found! Check the path")
        return False

def install_dependencies():
    """Install required packages."""
    packages = [
        "pip install ultralytics -q",
        "pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q",
        "pip install opencv-python matplotlib numpy pillow tqdm psutil -q"
    ]

    for pkg in packages:
        try:
            subprocess.run(pkg, shell=True, check=True)
            print(f"Installed: {pkg.split()[2]}")
        except subprocess.CalledProcessError:
            print(f"Failed to install: {pkg.split()[2]}")

    print("All dependencies installed!")

if __name__ == "__main__":
    print("Setting up Rice Leaf Disease Detection project...")
    mount_drive()
    check_dataset()
    install_dependencies()
    print("Setup complete!")

In [ ]:
%%writefile train.py
import os
import shutil
import time
from ultralytics import YOLO
from config import *

def train_model():
    """Train YOLOv8 model for Rice Leaf Disease Detection."""
    print(f"Starting training with {MODEL_BASE}...")
    start_time = time.time()

    # load the base model
    try:
        model = YOLO(MODEL_BASE)
        print("Base model loaded successfully")
    except Exception as e:
        print(f"Failed to load model: {e}")
        return False

    # train the model
    try:
        results = model.train(
            data=DATA_YAML,
            epochs=EPOCHS,
            imgsz=IMAGE_SIZE,
            batch=BATCH_SIZE,
            device=DEVICE,
            workers=WORKERS,
            project=PROJECT_DIR,
            name=EXPERIMENT_NAME,
            cache=CACHE
        )
        print(f"Training completed in {(time.time() - start_time)/60:.2f} minutes!")
    except Exception as e:
        print(f"Training failed: {e}")
        return False

    return True

def save_model():
    """Copy trained model to Google Drive."""
    try:
        source_dir = os.path.join(PROJECT_DIR, EXPERIMENT_NAME)
        dest_dir = os.path.join(SAVE_DIR, EXPERIMENT_NAME)

        if os.path.exists(source_dir):
            if os.path.exists(dest_dir):
                shutil.rmtree(dest_dir)

            shutil.copytree(source_dir, dest_dir)
            print(f"Model saved to: {dest_dir}")
            return True
        else:
            print(f"Source directory not found: {source_dir}")
            return False
    except Exception as e:
        print(f"Failed to save model: {e}")
        return False

if __name__ == "__main__":
    print("Rice Leaf Disease Detection - Training")
    print(f"Training for {EPOCHS} epochs with batch size {BATCH_SIZE}")

    if train_model():
        save_model()
        print("Training complete! Ready for prediction.")
    else:
        print("Training failed. Check logs for details.")

In [ ]:
%%writefile predict.py
import os
from ultralytics import YOLO
from config import *

def load_model():
    """Load the trained model."""
    try:
        model = YOLO(BEST_MODEL_PATH)
        print(f"Loaded model from: {BEST_MODEL_PATH}")
        return model
    except Exception as e:
        print(f"Failed to load model: {e}")
        return None

def run_prediction(model):
    """Run predictions on test images."""
    if not model:
        return False

    print(f"Running prediction on images in: {TEST_IMAGES}")
    try:
        # thực hiện dự đoán trên ảnh test
        results = model.predict(
            source=TEST_IMAGES,
            conf=CONFIDENCE,     # chỉ hiển thị kết quả có độ tin cậy cao
            save=True,           # lưu kết quả
            project=PRED_PROJECT,
            name=PRED_NAME
        )

        # đếm số lượng phát hiện theo từng loại bệnh
        class_counts = {}
        for r in results:
            if hasattr(r, 'boxes') and hasattr(r.boxes, 'cls'):
                for cls in r.boxes.cls:
                    cls_id = int(cls.item())
                    cls_name = model.names[cls_id]
                    class_counts[cls_name] = class_counts.get(cls_name, 0) + 1

        print(f"Processed {len(results)} images!")
        print("Detection results:")
        for cls_name, count in class_counts.items():
            print(f"  - {cls_name}: {count} detections")

        return True
    except Exception as e:
        print(f"Prediction failed: {e}")
        return False

def save_predictions():
    """Copy prediction results to Google Drive."""
    try:
        source_dir = os.path.join(PRED_PROJECT, PRED_NAME)
        dest_dir = os.path.join(SAVE_DIR, PRED_NAME)

        if os.path.exists(source_dir):
            if os.path.exists(dest_dir):
                import shutil
                shutil.rmtree(dest_dir)

            import shutil
            shutil.copytree(source_dir, dest_dir)
            print(f"Results saved to: {dest_dir}")
            return True
        else:
            print(f"Results directory not found: {source_dir}")
            return False
    except Exception as e:
        print(f"Failed to save results: {e}")
        return False

if __name__ == "__main__":
    print("Rice Leaf Disease Detection - Prediction")
    model = load_model()

    if run_prediction(model):
        save_predictions()
        print("Prediction complete! Check results for details.")
    else:
        print("Prediction failed. Check logs for details.")


In [ ]:
%%writefile main.py
import os
import time
import argparse

def print_header(text):
    print("\n" + "="*50)
    print(f"{text}")
    print("="*50)

def main():
    parser = argparse.ArgumentParser(description="Rice Leaf Disease Detection")
    parser.add_argument("--skip-setup", action="store_true", help="Skip setup steps")
    parser.add_argument("--skip-train", action="store_true", help="Skip training")
    parser.add_argument("--skip-predict", action="store_true", help="Skip prediction")
    args = parser.parse_args()

    start_time = time.time()

    # Setup
    if not args.skip_setup:
        print_header("SETUP")
        from setup import mount_drive, check_dataset, install_dependencies

        mount_drive()
        if not check_dataset():
            print("Cannot continue without dataset. Please check the path.")
            return
        install_dependencies()

    # Training
    if not args.skip_train:
        print_header("TRAINING")
        from train import train_model, save_model

        if train_model():
            save_model()
        else:
            print("Training failed. Cannot continue to prediction.")
            return

    # Prediction
    if not args.skip_predict:
        print_header("PREDICTION")
        from predict import load_model, run_prediction, save_predictions

        model = load_model()
        if run_prediction(model):
            save_predictions()

    # Done
    total_time = (time.time() - start_time) / 60
    print_header("COMPLETED")
    print(f"Total time: {total_time:.2f} minutes")
    print("Rice Leaf Disease Detection completed successfully!")

if __name__ == "__main__":
    print("RICE LEAF DISEASE DETECTION")
    print("YOLOv8 Object Detection Project")
    main()


In [ ]:
import os

# Đường dẫn thư mục dữ liệu
data_root = "/content/drive/My Drive/RiceLeafDisease"

# Kiểm tra thư mục chính
if os.path.exists(data_root):
    print(f"✅ Thư mục chính tồn tại: {data_root}")

    # Kiểm tra file data.yaml
    if os.path.exists(os.path.join(data_root, "data.yaml")):
        print("✅ File data.yaml tồn tại")
    else:
        print("❌ THIẾU file data.yaml")

    # Kiểm tra thư mục train, valid, test
    for folder in ["train", "valid", "test"]:
        folder_path = os.path.join(data_root, folder)
        if os.path.exists(folder_path):
            print(f"✅ Thư mục {folder} tồn tại")

            # Kiểm tra thư mục images
            images_path = os.path.join(folder_path, "images")
            if os.path.exists(images_path):
                num_images = len([f for f in os.listdir(images_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
                print(f"   - Có {num_images} ảnh trong thư mục {folder}/images")
            else:
                print(f"❌ THIẾU thư mục {folder}/images")
        else:
            print(f"❌ THIẾU thư mục {folder}")
else:
    print(f"❌ THIẾU thư mục chính: {data_root}")
    print("tạo thư mục RiceLeafDisease trong Google Drive")

In [ ]:
from setup import mount_drive, check_dataset, install_dependencies

print("==== THIẾT LẬP DỰ ÁN ====")
mount_drive()
check_dataset()
install_dependencies()
print("Hoàn tất thiết lập!")

In [ ]:
from train import train_model, save_model

print("==== BẮT ĐẦU HUẤN LUYỆN ====")
if train_model():
    save_model()
    print("Huấn luyện hoàn tất và lưu mô hình thành công!")
else:
    print("Huấn luyện thất bại. Vui lòng kiểm tra lỗi.")

In [ ]:
from predict import load_model, run_prediction, save_predictions

print("==== BẮT ĐẦU DỰ ĐOÁN ====")
model = load_model()
if model is not None:
    if run_prediction(model):
        save_predictions()
        print("Dự đoán hoàn tất và lưu kết quả thành công!")
    else:
        print("Dự đoán thất bại. Vui lòng kiểm tra lỗi.")
else:
    print("Không thể tải mô hình. Vui lòng kiểm tra đường dẫn.")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import random

# Đường dẫn đến thư mục kết quả dự đoán
pred_dir = "runs/predict/test_results-obb-100"

if os.path.exists(pred_dir):
    # Tìm tất cả ảnh kết quả
    pred_images = glob.glob(f"{pred_dir}/*.jpg")

    # Nếu có ít nhất 1 ảnh kết quả
    if pred_images:
        # Chọn ngẫu nhiên tối đa 4 ảnh để hiển thị
        sample_images = random.sample(pred_images, min(4, len(pred_images)))

        # Hiển thị các ảnh
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        axes = axes.flatten()

        for i, img_path in enumerate(sample_images):
            img = mpimg.imread(img_path)
            axes[i].imshow(img)
            axes[i].set_title(f"Kết quả dự đoán {i+1}", fontsize=12)
            axes[i].axis('off')

        # Ẩn các trục thừa
        for j in range(len(sample_images), 4):
            axes[j].axis('off')

        plt.tight_layout()
        plt.show()

        print(f"Đã hiển thị {len(sample_images)} ảnh kết quả dự đoán từ {pred_dir}")
    else:
        print(f"Không tìm thấy ảnh kết quả trong {pred_dir}")
else:
    print(f"Không tìm thấy thư mục kết quả: {pred_dir}")
    print("Vui lòng chạy quá trình dự đoán trước.")

Đánh giá hiệu suất của mô hình trên tập validation

In [ ]:
!python -c "from ultralytics import YOLO; model = YOLO('runs/train/exp-obb-100/weights/best-obb.pt'); model.val()"